In [10]:
import mysql.connector
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = ""
)
mycursor = mydb.cursor(buffered=True)
mycursor.execute("CREATE DATABASE bus_data")
mycursor.execute("USE bus_data")

In [11]:
import pandas as pd
df = pd.read_csv(r"F:\Guvi\Projects\Uploading\corrected_bus_types.csv")

In [13]:
df

,route_name,route_link,busname,bustype,departing_time,duration,reaching_time,star_rating,price,seats_available
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3749,Non-AC Semi-Sleeper,08:50,06h 55m,15:45,2.3,469.0,33
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 9357,Non-AC Semi-Sleeper,09:15,07h 35m,16:50,2.1,448.0,33
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 40569,Non-AC Semi-Sleeper,09:40,07h 20m,17:00,3.5,448.0,34
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3634,AC Semi-Sleeper,10:10,05h 50m,16:00,3.8,648.0,41
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 23508,AC Seater,10:30,05h 15m,15:45,3.9,528.0,38
...,...,...,...,...,...,...,...,...,...,...
11242,Digha to Habra,https://www.redbus.in/bus-tickets/digha-to-habra,SANTOSH BUS SERVICE,AC Seater,11:25,17h 50m,05:15,4.9,272.0,28
11243,Digha to Habra,https://www.redbus.in/bus-tickets/digha-to-habra,City Express,AC Seater,11:35,06h 10m,17:45,3.6,320.0,22
11244,Digha to Habra,https://www.redbus.in/bus-tickets/digha-to-habra,Satya Paribahan,AC Seater,11:50,06h 10m,18:00,4.6,320.0,21
11245,Digha to Habra,https://www.redbus.in/bus-tickets/digha-to-habra,Snemita Paribahan,AC Seater,11:35,06h 20m,17:55,2.9,370.0,15


In [ ]:
def inserting_mysql(details):
    """
    Creates a 'bus_details' table and 
    inserts all scraped bus details into the database.
    """
    try:
        # Create table if not exists
        mycursor.execute("""
        CREATE TABLE bus_details (
            id INT AUTO_INCREMENT PRIMARY KEY,
            route_name TEXT NOT NULL,
            route_link TEXT NOT NULL,
            bus_name TEXT NOT NULL, 
            bus_type TEXT NOT NULL,
            departing_time TIME,
            duration TEXT NOT NULL,
            reaching_time TIME,
            star_rating FLOAT NULL,
            price DECIMAL(10,2) NULL,
            seats_available INT NULL
        )
        """)

        # Insert each bus route detail
        for i in details:
            mycursor.execute("""
            INSERT INTO bus_details (
                route_name, route_link,
                bus_name, bus_type,
                departing_time, duration, reaching_time,
                star_rating, price,
                seats_available
            ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
            """,
            (
                i['route_name'], i['route_link'],
                i['busname'], i['bustype'],
                i['departing_time'], i['duration'], i['reaching_time'],
                i['star_rating'], i['price'], 
                i['seats_available']
            )
            )
        mydb.commit()

    except mysql.connector.Error as e:
        print(f"inserting_mysql Error: {e}")

final_details = df.to_dict('records')
inserting_mysql(final_details)